In [3]:
#线程和进程
#单线程
from random import randint
from time import time, sleep


def download_task(filename):
    print('开始下载%s...' % filename)
    time_to_download = randint(1, 6)
    sleep(time_to_download)
    print('%s下载完成! 耗费了%d秒' % (filename, time_to_download))


def main():
    start = time()
    download_task('Python从入门到住院.pdf')
    download_task('Peking Hot.avi')
    end = time()
    print('总共耗费了%.2f秒.' % (end - start))


if __name__ == '__main__':
    main()

开始下载Python从入门到住院.pdf...
Python从入门到住院.pdf下载完成! 耗费了10秒
开始下载Peking Hot.avi...
Peking Hot.avi下载完成! 耗费了8秒
总共耗费了18.00秒.


In [5]:
# 多线程
# 通过process方法实现多线程。Windows下不支持
from multiprocessing import Process
from os import getpid
from random import randint
from time import time, sleep


def download_task(filename):
    print('启动下载进程，进程号[%d].' % getpid())
    print('开始下载%s...' % filename)
    time_to_download = randint(5, 10)
    sleep(time_to_download)
    print('%s下载完成! 耗费了%d秒' % (filename, time_to_download))


def main():
    start = time()
    p1 = Process(target=download_task, args=('Python从入门到住院.pdf', ))
    p1.start()
    p2 = Process(target=download_task, args=('Peking Hot.avi', ))
    p2.start()
    p1.join()
    p2.join()
    end = time()
    print('总共耗费了%.2f秒.' % (end - start))


if __name__ == '__main__':
    main()

总共耗费了0.11秒.


In [7]:
# 线程的执行次数
# 下面的函数并不能控制两个线程合计执行10次。
# 要解决这个问题比较简单的办法是使用multiprocessing模块中的Queue类
from multiprocessing import Process
from time import sleep

counter = 0


def sub_task(string):
    global counter
    while counter < 10:
        print(string, end='', flush=True)
        counter += 1
        sleep(0.01)

        
def main():
    Process(target=sub_task, args=('Ping', )).start()
    Process(target=sub_task, args=('Pong', )).start()


if __name__ == '__main__':
    main()

In [9]:
#多线程
# 通过Thread 实现多线程
from random import randint
from threading import Thread
from time import time, sleep


def download(filename):
    print('Begin to download %s' % filename)
    elaps_time=randint(3,8)
    sleep(elaps_time)
    print('download completed,has consumed %d seconds' % elaps_time)


def main():
    start=time()
    #start = time()
    t1 = Thread(target=download, args=('Python从入门到住院.pdf',))
    t1.start()
    t2 = Thread(target=download, args=('Peking Hot.avi',))
    t2.start()
    t1.join()
    t2.join()
    end = time()
    print('总共耗费了%.3f秒' % (end - start))


if __name__ == '__main__':
    main()

Begin to download Python从入门到住院.pdf
Begin to download Peking Hot.avi
download completed,occupy 3 seconds
download completed,occupy 6 seconds
总共耗费了6.004秒


In [12]:
#多线程
# 通过 继承 Thread 类下载
from random  import randint
from threading import Thread
import time as tm
class DownloadTask(Thread):
    def __init__(self,filename):
        super().__init__()
        self._filename=filename
    def run(self):
        print('Begin to download %s' % self._filename)
        elaps_time=randint(3,8)
        sleep(elaps_time)
        print('%s download completed,has consumed %d seconds' % (self._filename,elaps_time))
def main():
    start=time()
    t1=DownloadTask('Python从入门到住院.pdf')
    t2=DownloadTask('how to pass a interview.pdf')
    t1.start()
    t2.start()
    t1.join()
    t2.join()
    end=time()
    print(f'total time is {end-start}')
main()

Begin to download Python从入门到住院.pdf
Begin to download how to pass a interview.pdf
how to pass a interview.pdf download completed,has consumed 7 seconds
Python从入门到住院.pdf download completed,has consumed 8 seconds
total time is 8.003484725952148


In [ ]:
# 线程锁
from time import sleep
from threading import Thread,Lock
#Define Account class
class Account(object):
    def __init__(self) :
        self._balance=0
        self._lock=Lock()
    def deposit(self,money):
        self._lock.acquire()
        try:
            new_balance=self._balance+money
            #sleep(0.01)
            self._balance=new_balance
        except Exception as e:
            print(e)
        finally:
            self._lock.release()  #除了bool方法，都要加括号
    @property
    def balance(self):
        return self._balance
class AddMoneyThread(Thread):
    def __init__(self,account,money):
        super().__init__()
        self._account=account
        self._money=money
    def run(self):
        self._account.deposit(self._money)   
def main():
    account=Account()
    threads=[]
    for _ in range(100):
        t=AddMoneyThread(account,1)
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    print('账户余额为: ￥%d元' % account.balance)
main()
            

账户余额为: ￥100元


# 应用案例
## 例子1：将耗时间的任务放到线程中以获得更好的用户体验。
如下所示的界面中，有“下载”和“关于”两个按钮，用休眠的方式模拟点击“下载”按钮会联网下载文件需要耗费10秒的时间，如果不使用“多线程”，我们会发现，当点击“下载”按钮后整个程序的其他部分都被这个耗时间的任务阻塞而无法执行了，这显然是非常糟糕的用户体验，代码如下所示。

In [2]:
import time
import tkinter
import tkinter.messagebox


def download():
    # 模拟下载任务需要花费10秒钟时间
    time.sleep(10)
    tkinter.messagebox.showinfo('提示', '下载完成!')


def show_about():
    tkinter.messagebox.showinfo('关于', '作者: 骆昊(v1.0)')


def main():
    top = tkinter.Tk()
    top.title('单线程')
    top.geometry('200x150')
    top.wm_attributes('-topmost', True)

    panel = tkinter.Frame(top)
    button1 = tkinter.Button(panel, text='下载', command=download)
    button1.pack(side='left')
    button2 = tkinter.Button(panel, text='关于', command=show_about)
    button2.pack(side='right')
    panel.pack(side='bottom')

    tkinter.mainloop()


if __name__ == '__main__':
    main()

: 

: 

In [ ]:
# 修改后代码
import time
import tkinter
import tkinter.messagebox
from threading import Thread


def main():

    class DownloadTaskHandler(Thread):

        def run(self):
            time.sleep(10)
            tkinter.messagebox.showinfo('提示', '下载完成!')
            # 启用下载按钮
            button1.config(state=tkinter.NORMAL)

    def download():
        # 禁用下载按钮
        button1.config(state=tkinter.DISABLED)
        # 通过daemon参数将线程设置为守护线程(主程序退出就不再保留执行)
        # 在线程中处理耗时间的下载任务
        DownloadTaskHandler(daemon=True).start()

    def show_about():
        tkinter.messagebox.showinfo('关于', '作者: 骆昊(v1.0)')

    top = tkinter.Tk()
    top.title('单线程')
    top.geometry('200x150')
    top.wm_attributes('-topmost', 1)

    panel = tkinter.Frame(top)
    button1 = tkinter.Button(panel, text='下载', command=download)
    button1.pack(side='left')
    button2 = tkinter.Button(panel, text='关于', command=show_about)
    button2.pack(side='right')
    panel.pack(side='bottom')

    tkinter.mainloop()


if __name__ == '__main__':
    main()



## 例子2：使用多进程对复杂任务进行“分而治之”。
此处使用Thread

In [1]:
from time import time


def main():
    total = 0
    number_list = [x for x in range(1, 100000001)]
    start = time()
    for number in number_list:
        total += number
    print(total)
    end = time()
    print('Execution time: %.3fs' % (end - start))


if __name__ == '__main__':
    main()

5000000050000000
Execution time: 2.665s


In [5]:
from time import time
from threading import Thread
def task_handler(curr_list, result_list):
    total = 0
    for number in curr_list:
        total += number
    result_list.append(total)
def main():
    threads=[]
    number_list = [x for x in range(1, 100000001)]
    index = 0
    result_list=[]
    for _ in range(8):
        p=Thread(target=task_handler,args=(number_list[index:index + 12500000],result_list))
        index+=12500000
        p.start()
        threads.append(p)
    start = time()
    for thread in threads:
        thread.join()
    total = 0
    for i in result_list:
        total += i
    print(total)
    end = time()
    print('Execution time: ', (end - start), 's', sep='')


if __name__ == '__main__':
    main()    

    

5000000050000000
Execution time: 1.9796671867370605s
